In [ ]:
from src.llm_plan.llm import GPT_OSS

system_prompt = "You are a useful assistant."
prompt = "Tell me a joke."

model = GPT_OSS(model_name="openai/gpt-oss-120b", map="cpu", reasoning="Low")

model.generate_sync(system_prompt, prompt)

In [ ]:
prompt = "What is the capital of France?"
model.generate_sync(system_prompt, prompt)

In [1]:
import inspect
from src.llm_plan.llm import ChatGPT

model = ChatGPT("gpt-5-mini")
system_prompt = "You are a helpful assistant."
prompt = inspect.cleandoc("""\
As a logistics coordinator for a multi-robot system, your objective is to find a common maintenance window. Four autonomous units—M, J, E, and T—must synchronize for a mandatory diagnostics check that takes exactly 30 minutes. A viable window is guaranteed to exist.

The operational parameters for the system on Monday are between 09:00 and 17:00. Below are the logs of each unit's pre-scheduled, independent tasks, during which they cannot be interrupted.

Unit M is engaged in processing tasks during Monday's intervals: 11:00 to 11:30, and 15:00 to 16:30.

Unit J is executing its primary functions on Monday from 12:30 to 13:00, and 13:30 to 15:00.

Unit E has scheduled activities on Monday during these times: 10:00 to 13:00, 14:30 to 15:00, and 16:00 to 16:30.

Unit T is occupied with its routines on Monday at: 09:30 to 10:30, 12:00 to 13:00, 13:30 to 14:00, 14:30 to 15:00, 15:30 to 16:00, and 16:30 to 17:00.

Furthermore, a specific protocol for Unit T requires that its diagnostics check must be initiated and concluded before the 12:00 midday system reset on Monday.

Determine a time slot that satisfies the operational availability of all four units and adheres to all specified constraints.
""")
response = model.generate_sync(system_prompt, prompt)
print(response)

/home/emanuele/MultiAgentPlanning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Schedule the 30-minute diagnostics for Monday 09:00–09:30.

Reason: 
- T is free 09:00–09:30 (it is busy 09:30–10:30 and must finish before 12:00).  
- E is free 09:00–10:00 (busy 10:00–13:00).  
- M is free 09:00–11:00 (busy 11:00–11:30, later 15:00–16:30).  
- J is free 09:00–12:30 (busy beginning 12:30).  

The only common 30-minute slot before T’s 12:00 constraint is 09:00–09:30.


In [1]:
import re
import random
import copy
import json
from pathlib import Path

# regex to catch times like 9:00, 09:00, 15:30
TIME_PATTERN = re.compile(r"\b(\d{1,2}):(\d{2})\b")

# regex to catch AM/PM times like 2:30 PM, 11:00 AM
AMPM_PATTERN = re.compile(r"\b(\d{1,2}):(\d{2}) ?([AP]M)\b", re.IGNORECASE)

def shift_times_in_text(text: str, shift: int) -> str:
    def replace_24h(match):
        hour, minute = int(match.group(1)), int(match.group(2))
        new_hour = (hour + shift) % 24
        return f"{new_hour}:{minute:02d}"

    def replace_ampm(match):
        hour, minute, ampm = int(match.group(1)), int(match.group(2)), match.group(3).upper()
        # Convert to 24h first
        if ampm == "PM" and hour != 12:
            hour += 12
        if ampm == "AM" and hour == 12:
            hour = 0
        # shift
        new_hour = (hour + shift) % 24
        # convert back to AM/PM
        if new_hour == 0:
            display_hour = 12
            new_ampm = "AM"
        elif 1 <= new_hour < 12:
            display_hour = new_hour
            new_ampm = "AM"
        elif new_hour == 12:
            display_hour = 12
            new_ampm = "PM"
        else:
            display_hour = new_hour - 12
            new_ampm = "PM"
        return f"{display_hour}:{minute:02d} {new_ampm}"

    text = TIME_PATTERN.sub(replace_24h, text)
    text = AMPM_PATTERN.sub(replace_ampm, text)
    return text

def shift_calendar_json_file(input_file: str, output_file: str):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    shifted_data = copy.deepcopy(data)
    shift = 10  # consistent random shift

    for key, value in shifted_data.items():
        if "prompt_0shot" in value:
            value["prompt_0shot"] = shift_times_in_text(value["prompt_0shot"], shift)
        if "response" in value:
            value["response"] = shift_times_in_text(value["response"], shift)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(shifted_data, f, indent=2, ensure_ascii=False)

    return output_file

# Example usage:
shift_calendar_json_file("./data/miscellanea/calendar_easy_to_hard_analogy.json",
                         "./data/miscellanea/calendar_easy_to_hard_analogy_shifted.json")


'./data/miscellanea/calendar_easy_to_hard_analogy_shifted.json'

In [2]:
import re
import json
import sys

def convert_file(input_filename, output_filename):
    """
    Reads a JSON file in scheduling format, converts it, and writes to a new JSON file.
    """
    # 1. Read the input file
    try:
        with open(input_filename, 'r') as f:
            input_data = json.load(f)
    except FileNotFoundError:
        sys.exit(f"Error: Input file not found at '{input_filename}'")
    except json.JSONDecodeError:
        sys.exit(f"Error: The file '{input_filename}' is not a valid JSON file.")

    # 2. Process the data
    converted_dataset = {}
    for key, value in input_data.items():
        original_prompt_text = value.get('prompt_0shot', '')
        task_text_match = re.search(r"TASK:(.*?)SOLUTION:", original_prompt_text, re.DOTALL)
        
        if not task_text_match:
            continue

        task_text = task_text_match.group(1).strip()

        # --- Parsing Logic ---
        main_match = re.search(r"schedule a meeting for (.*?) for (one hour|half an hour)", task_text)
        if not main_match:
            continue

        names_str = main_match.group(1)
        names = [name.strip() for name in re.split(r',\s*|\s+and\s+', names_str)]
        initials = [name[0] for name in names]
        duration_map = {"one hour": "1 hour", "half an hour": "30 minutes"}
        duration = duration_map.get(main_match.group(2), "an unknown duration")

        schedules = {}
        for name in names:
            schedule_match = re.search(fr"^{name}.*? on Monday during (.*?);", task_text, re.MULTILINE)
            schedules[name] = schedule_match.group(1).strip() if schedule_match else "free all day"
        
        pref_match = re.search(r"(\w+) would (?:like to avoid more meetings|rather not meet) on Monday (after|before) (\d{1,2}:\d{2})", task_text)
        earliest_match = re.search(r"You would like to schedule the meeting at their earlist availability", task_text)
        
        # --- Building the New Prompt ---
        num_units_word = {1: "one", 2: "two", 3: "three", 4: "four"}.get(len(names), str(len(names)))
        
        intro = (
            f"As a logistics coordinator for a multi-robot system, your objective is to find a common maintenance window. "
            f"{num_units_word.title()} autonomous units—{', '.join(initials)}—must synchronize for a mandatory diagnostics check that takes "
            f"exactly {duration}. A viable window is guaranteed to exist."
        )

        time_window = "The operational parameters for the system on Monday are between 09:00 and 17:00. Below are the logs of each unit's pre-scheduled, independent tasks, during which they cannot be interrupted."
        
        schedule_lines = []
        for name in names:
            if schedules[name] == "free all day":
                schedule_lines.append(f"* **Unit {name[0]}** has an open operational window for the entire day.")
            else:
                schedule_lines.append(f"* **Unit {name[0]}** is engaged in processing tasks during Monday's intervals: {schedules[name]}.")
        schedules_block = "\n".join(schedule_lines)
        
        preference_block = ""
        if earliest_match:
            preference_block = "A key operational directive is to schedule this check at the earliest possible opportunity to maximize system uptime."
        elif pref_match:
            pref_name, condition, pref_time = pref_match.groups()
            condition_text = "initiated and concluded before" if condition == 'before' else f"initiated after"
            preference_block = f"Furthermore, a specific protocol for **Unit {pref_name[0]}** requires that its diagnostics check must be {condition_text} the {pref_time} system reset on Monday."

        cta = f"Determine a time slot that satisfies the operational availability of all {num_units_word} units and adheres to all specified constraints."
        
        converted_prompt = "\n\n".join(filter(None, [intro, time_window, schedules_block, preference_block, cta]))
        
        # Add to our results
        new_key = key.replace("calendar_scheduling", "robot_logistics")
        converted_dataset[new_key] = {
            "original_prompt_text": task_text,
            "prompt_0shot": converted_prompt,
            "golden_plan": value.get("golden_plan", "N/A")
        }

    # 3. Write the output file
    with open(output_filename, 'w') as f:
        json.dump(converted_dataset, f, indent=2)
    
    print(f"✅ Success! File converted and saved as '{output_filename}'")


# --- Main execution block ---
if __name__ == "__main__":
    # Define your input and output files here
    INPUT_JSON_FILE = "./data/miscellanea/calendar_easy_to_hard.json"
    OUTPUT_JSON_FILE = "./data/miscellanea/calendar_easy_to_hard_analogy.json"
    
    # Run the conversion
    convert_file(INPUT_JSON_FILE, OUTPUT_JSON_FILE)

✅ Success! File converted and saved as './data/miscellanea/calendar_easy_to_hard_analogy.json'
